<a href="https://colab.research.google.com/github/rutia2044/rutiarbiv-gmail.com/blob/master/VGG_imagnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWi1i2OxrcU0lWfSCGjqf6ICkJ-_7KEFNWG7c5vlaA8oVHY0PFr5MxE
Mounted at /content/drive


In [2]:
import numpy as np
 
import torchvision.transforms as transforms
from torchvision.utils import make_grid
 
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
 
import torch.nn as nn
import torch.nn.functional as F
import torch
 
import os
import glob
 
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
from tensorflow import keras,summary


In [3]:
#import numpy as np
#import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from keras.applications.vgg16 import VGG16

#from keras import models, layers,applications


In [4]:
images = []
root = '/content/drive/MyDrive/gan-getting-started/monet_jpg'
for file in tqdm(os.listdir(root)):
    images.append(plt.imread(root+'/'+file))
images = np.array(images)

100%|██████████| 300/300 [01:46<00:00,  2.82it/s]


In [5]:
images = images/255

In [6]:
print(images[0][0][0][0])

0.6980392156862745


In [7]:

#Encoder

EncoderBaseModel = VGG16(include_top=False,input_shape=(256, 256, 3),weights='imagenet')
EncoderBaseModel.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

In [13]:
ImgPred = EncoderBaseModel.predict(images)
ImgPred.shape

(300, 8, 8, 512)

In [ ]:
from sklearn.manifold import TSNE

#tsneAllImages = TSNE(n_components=700).fit_transform( ImgPred.reshape(300,-1))
tsneAllImages = TSNE(n_components=7000,method='exact').fit_transform( ImgPred.reshape(300,-1))
print(tsneAllImages.shape)

In [ ]:

# save numpy array as csv file
#from numpy import asarray
from numpy import savetxt
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to csv file
savetxt('VGG_TSNE.csv', tsneAllImages, delimiter=',')

In [ ]:

# load numpy array from csv file
from numpy import loadtxt
# load array
tsneAllImages = loadtxt('VGG_TSNE.csv', delimiter=',')
# print the array
print(tsneAllImages)

In [ ]:
v1=ImgPred[0:30,:,:,:]
v2=ImgPred[30:60,:,:,:]
v3=ImgPred[30:60,:,:,:]
v4=ImgPred[30:60,:,:,:]
v5=ImgPred[30:60,:,:,:]
v1.shape

(30, 8, 8, 512)

FID

In [ ]:
# example of calculating the frechet inception distance
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
 
# calculate frechet inception distance
def calculate_fid(act1, act2):
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
 


In [ ]:
# define two collections of activations
act1 = random(30*7000)
act1 = act1.reshape((30,7000))
act2 = random(30*7000)
act2 = act2.reshape((30,7000))
# fid between act1 and act1
fid = calculate_fid(act1, act1)
print('FID (same): %.3f' % fid)
# fid between act1 and act2
fid = calculate_fid(act1, act2)
print('FID (different): %.3f' % fid)

FID (same): -0.001
FID (different): 1142.155


In [ ]:
from sklearn.manifold import TSNE

#tsneAllImages = TSNE(n_components=700,method='exact').fit_transform( ImgPred.reshape(300,-1))
tsneAllImages = TSNE(n_components=7000,method='exact').fit_transform( ImgPred.reshape(300,-1))
print(tsneAllImages.shape)


(300, 7000)


In [ ]:
tsneV1_Images=tsneAllImages[0:30,:]
tsneV2_Images=tsneAllImages[30:60,:]

tsneV1_Images.shape

(30, 7000)

In [ ]:
tsneV1_Images = TSNE(n_components=700,method='exact').fit_transform( v1.reshape(30,-1))
#tsneV1_Images = TSNE(n_components=3).fit_transform( v1.reshape(30,-1))
print(tsneV1_Images.shape)
tsneV2_Images = TSNE(n_components=700,method='exact').fit_transform( v2.reshape(30,-1))
#tsneV2_Images = TSNE(n_components=3).fit_transform( v2.reshape(30,-1))
print(tsneV2_Images.shape)

In [ ]:
fid = calculate_fid(tsneAllImages, tsneAllImages)
print('FID (same): %.3f' % fid)

fid = calculate_fid(tsneAllImages, tsneV1_Images)
print('FID (different): %.3f' % fid)

fid = calculate_fid(tsneAllImages, tsneV2_Images)
print('FID (different): %.3f' % fid)

FID (same): -0.002
FID (different): 422.071
FID (different): 411.075


In [ ]:


v1 = v1.reshape((30,32768))
v2 = v2.reshape((30,32768))
fid = calculate_fid(v1, v1)
print('FID (same): %.3f' % fid)

fid = calculate_fid(v1, v2)
print('FID (different): %.3f' % fid)

TSNE + CLUSTERING

In [ ]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2).fit_transform( ImgPred.reshape(300,-1))
tsne.shape

(300, 2)

In [ ]:
from sklearn.cluster import KMeans
import collections

In [ ]:
kmeans = KMeans(n_clusters=30).fit(tsne)
collections.Counter(kmeans.labels_)

Counter({0: 19,
         1: 11,
         2: 13,
         3: 9,
         4: 7,
         5: 7,
         6: 22,
         7: 7,
         8: 13,
         9: 10,
         10: 7,
         11: 20,
         12: 6,
         13: 4,
         14: 7,
         15: 10,
         16: 5,
         17: 4,
         18: 9,
         19: 8,
         20: 9,
         21: 11,
         22: 14,
         23: 5,
         24: 3,
         25: 14,
         26: 8,
         27: 15,
         28: 11,
         29: 12})

In [ ]:
select=[]
for i in range (30):
  select.append(np.nonzero(kmeans.labels_ == i)[0][0])
select

[10,
 29,
 9,
 7,
 3,
 75,
 4,
 40,
 14,
 57,
 8,
 11,
 32,
 2,
 45,
 41,
 13,
 97,
 17,
 6,
 1,
 26,
 27,
 109,
 34,
 0,
 115,
 20,
 15,
 37]

AutoEnc Base VGG